## Read Data
### Use Prompt Generation to generate code to read pandas data frame

In [1]:
# Assuming the dataframe already exists with the variable name 'df'
# We can use the pandas read_csv() function to read a CSV file and assign it to a new variable

import pandas as pd
fp = "../../data/online_retail_II.csv"
# Read the CSV file and assign it to a new variable
new_df = pd.read_csv(fp)

# Print the new dataframe
print(new_df)

        Invoice StockCode                          Description Quantity  \
0        489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS       12   
1        489434    79323P                   PINK CHERRY LIGHTS       12   
2        489434    79323W                  WHITE CHERRY LIGHTS       12   
3        489434     22041         RECORD FRAME 7" SINGLE SIZE        48   
4        489434     21232       STRAWBERRY CERAMIC TRINKET BOX       24   
...         ...       ...                                  ...      ...   
1067367  581587     22899         CHILDREN'S APRON DOLLY GIRL         6   
1067368  581587     23254        CHILDRENS CUTLERY DOLLY GIRL         4   
1067369  581587     23255      CHILDRENS CUTLERY CIRCUS PARADE        4   
1067370  581587     22138        BAKING SET 9 PIECE RETROSPOT         3   
1067371  581587      POST                              POSTAGE        1   

           InvoiceDate Price Customer ID         Country  
0         12/1/09 7:45  6.95     13085.0

/var/folders/fb/t_m5qpcj6qq85rvkh73vbxh40000gn/T/ipykernel_5612/4292484759.py:7: DtypeWarning: Columns (3,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  new_df = pd.read_csv(fp)


## Noise Filter #1 Definition
### Generated through prompt einblick extension

In [2]:


# Remove null values for columns "StockCode", "Customer ID" and "Description"
new_df.dropna(subset=["StockCode", "Customer ID", "Description"], inplace=True)



## Noise Filter #3 

In [3]:
def good_quantity_record_check(x):
    try:
        f_x = float(x)
        if f_x > 0:
            return True
        else:
            return False # returned purchase
    except :
        return False
good_quantity_records = new_df["Quantity"].apply(good_quantity_record_check)
new_df = new_df[good_quantity_records]
new_df = new_df.reset_index(drop=True)

## Noise Filter #4 Definition

In [4]:
def not_test_product(x):
    if "TEST" in x:
        return False
    elif x in ["ADJUST", "BANK CHARGES", "C2", "M"]:
        return False
    else:
        return True
valid_products = new_df["StockCode"].apply(not_test_product)
new_df = new_df[valid_products]
new_df = new_df.reset_index(drop=True)

## Noise Filter #4 Definition

In [5]:
attr_types = {"Invoice": str, "StockCode": str, "Description": str,\
             "Quantity": float, "InvoiceDate": 'datetime64[ns]', "Price": float,\
             "Customer ID": str, "Country": str}
new_df = new_df.astype(attr_types)
Q1_2010 = (new_df["InvoiceDate"].dt.year == 2010) & (new_df["InvoiceDate"].dt.quarter == 1)

/var/folders/fb/t_m5qpcj6qq85rvkh73vbxh40000gn/T/ipykernel_5612/116447840.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  new_df = new_df.astype(attr_types)


In [6]:
new_df = new_df[Q1_2010]
new_df = new_df.reset_index(drop=True)

In [7]:
new_df["item_total"] = new_df["Quantity"] * new_df["Price"]
dsbysc = new_df.groupby([new_df.InvoiceDate.dt.day_of_year, new_df.StockCode])
dsbysc = dsbysc["item_total"].sum().to_frame().reset_index()
dfQ1_PA = dsbysc.pivot(index="InvoiceDate", columns="StockCode", values="item_total").fillna(0)

In [8]:
dfQ1_PA

StockCode,10002,10120,10123C,10124A,10125,10133,10134,10135,10138,11001,...,90214L,90214M,90214N,90214O,90214P,90214R,90214S,90214V,PADS,POST
InvoiceDate,,,,,,,,,,,,,,,,,,,,,
4,2.55,6.3,0.0,0.0,0.0,0.00,0.00,1.25,0.00,0.00,...,0.0,0.0,2.5,0.0,0.00,0.0,1.25,0.0,0.0,0.0
5,0.00,0.0,0.0,0.0,0.0,0.00,0.00,12.50,0.00,3.38,...,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0
6,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0
7,0.00,0.0,0.0,0.0,0.0,108.00,212.00,0.00,0.00,27.04,...,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,19.0
8,10.20,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,477.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,11.05,0.0,0.0,0.0,0.0,0.00,0.00,25.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,15.0
87,0.00,0.0,0.0,0.0,5.1,0.85,1.25,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0
88,0.00,0.0,0.0,0.0,17.0,0.00,0.00,0.00,5.04,0.00,...,0.0,0.0,0.0,0.0,1.25,0.0,0.00,0.0,0.0,0.0
